# Lista 4 de Introdução à Programação e Ciência de Dados

Professor: Rafael de Pinho

Monitor: Sillas Rocha

Aluno: Iago Dantas Figueirêdo

## Instruções Gerais
- Cada exercício deve ser implementado em um módulo separado, com funções reutilizáveis.
- Documente todas as funções com docstrings no estilo PEP 257 e use type hints.
- Utilize apenas as bibliotecas padrão do Python e o NumPy. Bibliotecas como ```pandas```, ```scipy``` e similares não são permitidas.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from threading_basics import (
    simular_traders,
    simular_feeds_de_dados,
)

## Parte 1: Introdução ao Multithreading
Esta parte foca nos conceitos básicos de multithreading, demonstrando como criar e gerenciar threads em cenários financeiros simples. Implemente os exercícios no módulo ```threading_basics.py```.

### 1. Simulação de Traders Colocando Ordens Concorrentemente

**Função**: ```simular_traders(num_traders: int, num_ordens: int) -> Dict[str, List[Dict[str, Any]]]```

**Descrição**: Implemente uma simulação onde múltiplas threads (traders) inserem ordens de compra ou venda em uma estrutura compartilhada chamada ```order_book```, que consiste em dois dicionários: um para compras e um para vendas. Cada ordem deve conter um ID único, um preço e uma quantidade. Utilize ```threading.Lock``` para garantir que o acesso a ```order_book``` seja seguro e atômico. Após todas as threads finalizarem, retorne o estado final da estrutura ```order_book```.

**Parâmetros**:
- ```num_traders```: Número de threads (traders) a serem criadas.
- ```num_ordens```: Número de ordens que cada trader deve colocar.

**Retorno**: O estado final da estrutura ```order_book```, que é um dicionário com chaves 'buy' e 'sell', cada uma contendo uma lista de ordens (cada ordem é um dicionário com 'id', 'price' e 'quantity').

In [3]:
# Exemplo de uso da função simular_traders

num_traders = 3
num_orders = 4

order_book = simular_traders(num_traders, num_orders)

print("Livro de Ordens:")
for key in ["buy", "sell"]:
    print(f"  {key.capitalize()}:")
    for order in order_book[key]:
        print(f"    {order['id']}: {order['price']} ({order['quantity']})")

Livro de Ordens:
  Buy:
    trader_1_ordem_2: 84.39 (1000)
    trader_1_ordem_3: 96.11 (100)
    trader_2_ordem_1: 83.09 (600)
    trader_2_ordem_3: 96.87 (1000)
    trader_2_ordem_4: 96.06 (800)
  Sell:
    trader_0_ordem_1: 100.6 (500)
    trader_0_ordem_2: 104.54 (200)
    trader_0_ordem_3: 111.85 (100)
    trader_0_ordem_4: 101.96 (400)
    trader_1_ordem_1: 111.27 (900)
    trader_1_ordem_4: 113.99 (600)
    trader_2_ordem_2: 101.95 (200)


### 2. Simulação de Feeds de Dados Concorrentes

**Função**: ```simular_feeds_de_dados(acoes: List[str], tempo_total: int) -> Dict[str, float]```

**Descrição**: Crie um dicionário compartilhado ```prices``` que armazena os preços atuais de várias ações. Crie uma thread para cada ação em ```acoes```, representando um feed de dados que atualiza periodicamente (a cada $1-3$ segundos) o preço da sua ação no dicionário. Use um ```threading.Lock``` para sincronizar o acesso ao dicionário. Crie uma thread adicional que imprime os preços atuais a cada 5 segundos. A simulação deve rodar por ```tempo_total``` segundos. Retorne o dicionário final de preços após a simulação.

**Parâmetros**:
- ```acoes```: Lista de nomes de ações (e.g., ["AAPL", "GOOG", "TSLA"]).
- ```tempo_total```: Tempo total de simulação em segundos.

**Retorno**: O dicionário final de preços após a simulação.

In [4]:
# Exemplo de uso da função simular_feeds_de_dados

acoes = ["AAPL", "GOOG", "TSLA"]
tempo_total = 12  # segundos

feed_de_dados = simular_feeds_de_dados(acoes, tempo_total)

{'AAPL': 18.06, 'GOOG': 7.11, 'TSLA': 10.83}
{'AAPL': 18.75, 'GOOG': 7.24, 'TSLA': 11.22}
{'AAPL': 17.36, 'GOOG': 7.24, 'TSLA': 11.44}
